<a href="https://colab.research.google.com/github/Gaumis/Tcs_HumAIn/blob/master/Chatbot_complications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    print(json.dumps(o, indent=2))

In [76]:
import sys
python = sys.executable

!{python} -m pip install -U rasa_core==0.9.6 rasa_nlu[spacy]===0.12.3;

!{python} -m spacy download en_core_web_md
!{python} -m spacy link en_core_web_md en --force;

Requirement already up-to-date: rasa_core==0.9.6 in /usr/local/lib/python3.6/dist-packages (0.9.6)
Requirement already up-to-date: rasa_nlu[spacy]===0.12.3 in /usr/local/lib/python3.6/dist-packages (0.12.3)

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
import rasa_nlu
import rasa_core
import spacy

In [78]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
!breq install graphviz
!{python} -m pip install pygraphviz;

/bin/bash: breq: command not found


In [79]:
nlu_md = """
## intent:train
-i want to book a one way train
-I am looking for train
-I want to book a return train
-I want to book train
-can you help me with train booking?
-I want to go from Mumbai to Pune
-Looking for a oneway train
-Help me in booking train
-I want to book a train ticket
-i want to book a two way train

## intent:inform
-[DEL](location)
-[BOM](location)
-[CCU](location)
-[BLR](location)
-[HYD](location)
-[MAA](location)
-[PNQ](location)
-[IXC](location)
-[09-01-2019](date)
- I want for [09-02-2019](date)
- i want to travel on [21-04-2019](date)

## intent:affirmation
-ok
-yea
-yes
-definitely
-yah

## intent:deny
-no
-not yet
-never
-not now
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [80]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"           
- name: "tokenizer_spacy"             
- name: "ner_crf"                     
- name: "intent_featurizer_spacy"     
- name: "intent_classifier_sklearn"  
- name: "ner_synonyms"               
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


In [81]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))
interpreter = trainer.train(training_data)
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 30 (4 distinct intents)
	- Found intents: 'train', 'inform', 'affirmation', 'deny'
	- entity examples: 11 (2 distinct entities)
	- found entities: 'date', 'location'

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'


error: ignored

In [62]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 37.4MB 1.3MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.0.0-cp36-none-any.whl size=37405978 sha256=fd7ca1d1a066073edc448f74a1d04553638d39dc2e4e144023cebff73f87de83
  Stored in directory: /tmp/pip-ephem-wheel-cache-pwnywurd/wheels/54/7c/d8/f86364af8fbba7258e14adae115f18dd2c91552406edc3fdaa
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.1.0
    Uninstalling en-core-web-sm-2.1.0:
      Successfully uninstalled en-core-web-sm-2.1.0

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [63]:
!python -m spacy validate


    Installed models (spaCy v2.0.18)
    /usr/local/lib/python3.6/dist-packages/spacy

    TYPE        NAME                  MODEL                 VERSION                                   
    package     en-core-web-sm        en_core_web_sm        2.0.0    ✔      
    package     en-core-web-md        en_core_web_md        2.0.0    ✔      
    link        en_core_web_md        en_core_web_md        2.0.0    ✔      
    link        en_core_web_sm        en_core_web_sm        2.0.0    ✔      
    link        en                    en_core_web_md        2.0.0    ✔      


In [0]:
pprint(interpreter.parse("train status"))

In [0]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

In [51]:
stories_md = """
## fallback
- utter_unclear


## Story 1
* train
    - utter_boarding
* inform{"location": "BOM"}
    - action_save_origin
    - slot{"from": "BOM"}
    - utter_destination
* inform{"location": "DEL"}
    - action_save_destination
    - slot{"to": "DEL"}
    - utter_date
* inform{"date": "20-01-2019"}
    - action_save_date
    - slot{"date": "20-01-2019"}
    - utter_confirm
* affirmation
    - action_get_train
	- utter_check_another_one
* deny
	- utter_thanks
	- action_restart

## Stry 2-multiple steps
* train
    - utter_boarding
* inform{"location": "CCU"}
    - action_save_origin
    - slot{"from": "CCU"}
    - utter_destination
* inform{"location": "BLR"}
    - action_save_destination
    - slot{"to": "BLR"}
    - utter_date
* inform{"date": "03-02-2019"}
    - slot{"date": "03-02-2019"}
    - action_save_date
    - slot{"date": "03-02-2019"}
    - utter_confirm
* affirmation
    - action_get_train
    - utter_check_another_one
* affirmation
	- action_slot_reset
	- reset_slots
    - utter_boarding
* inform{"location": "BOM"}
    - action_save_origin
    - slot{"from": "BOM"}
    - utter_destination
* inform{"location": "DEL"}
    - action_save_destination
    - slot{"to": "DEL"}
    - utter_date
* inform{"date": "10-02-2019"}
    - slot{"date": "10-02-2019"}
    - action_save_date
    - slot{"date": "10-02-2019"}
    - utter_confirm
* affirmation
    - action_get_train
    - utter_check_another_one
* deny
    - utter_thanks
    - action_restart
"""
%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


In [52]:
domain_yml = """
intents:
- ticket
- inform
- affirmation
- deny

entities:
- location 
- date

slots:
   from:
            type: text
   to:
            type: text
   date:
            type: text

actions:
- utter_boarding
- utter_destination
- utter_date
- utter_confirm
- utter_check_another_one
- utter_thanks
- utter_unclear
- action_save_origin
- action_save_destination
- action_save_date
- action_get_train
- action_slot_reset
templates:
  
  utter_boarding:
  - text: "We'll help you find the latest train schedule. First, please provide your origin station code?"
  
  utter_destination:
  - text: "And the destination station code?"
  
  utter_date:
  - text: "What is the date for your travel(in dd-mm-yyyy)?"
  
  utter_confirm:
  - text: "I will be making inquiry for train from {from} to {to} on {date}. Is that correct?"
  
  utter_check_another_one:
  - text: "Do you want to make another inquiry?"
  
  utter_thanks:
  - text: "Thanks for contacting us. Have a good day!"
  
  utter_unclear:
  - text: "I am not sure what you are aiming for.Kindly try it again"
  """

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


In [18]:
!pip install rasa_core_sdk

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_core.domain import Domain
#from rasa_core.trackers import EventVerbosity

import logging
logger = logging.getLogger(__name__)

import requests
import json


from rasa_core.actions import Action
from rasa_core.events import SlotSet
#from rasa_core_sdk import Action
#from rasa_core_sdk.events import SlotSet
from rasa_core_sdk.events import UserUtteranceReverted
from rasa_core_sdk.events import AllSlotsReset
from rasa_core_sdk.events import Restarted


		 
class SaveOrigin(Action):
	def name(self):
		return 'action_save_origin'
		
	def run(self, dispatcher, tracker, domain):
		orig = next(tracker.get_latest_entity_values("location"), None)
		if not orig:
			dispatcher.utter_message("Please enter a valid station code")
			return [UserUtteranceReverted()]
		return [SlotSet('from',orig)]
	


class SaveDestination(Action):
	def name(self):
		return 'action_save_destination'
		
	def run(self, dispatcher, tracker, domain):
		dest = next(tracker.get_latest_entity_values("location"), None)
		if not dest:
			dispatcher.utter_message("Please enter a valid station code")
			return [UserUtteranceReverted()]
		return [SlotSet('to',dest)]
		
		
class SaveDate(Action):
	def name(self):
		return 'action_save_date'
		
	def run(self, dispatcher, tracker, domain):
		inp = next(tracker.get_latest_entity_values("date"), None)
		if not inp:
			dispatcher.utter_message("Please enter a valid date")
			return [UserUtteranceReverted()]
		return [SlotSet('date',inp)]
		
class ActionSlotReset(Action):
  def name(self):
    return 'action_slot_reset' 	
  def run(self, dispatcher, tracker, domain):
    return[AllSlotsReset()]
     
class SaveDate(Action):
  def name(self):
    return 'action_save_date'	
  def run(self, dispatcher, tracker, domain):
    dispatcher.utter_message("Please enter a valid date")
    return [UserUtteranceReverted()]
        

In [40]:

from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

agent = Agent('domain.yml', policies=[KerasPolicy()])
training_data = agent.load_data('stories.md')
agent.train(
        training_data,
        validation_split=0.0
)

agent.persist('models/dialogue')

ValueError: ignored

In [83]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

import logging
import tensorflow
import rasa_core
from rasa_core.agent import Agent
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter
#from rasa_core.run import serve_application
#from rasa_core.utils import EndpointConfig

logger = logging.getLogger(__name__)

fallback = FallbackPolicy(fallback_action_name="utter_unclear",core_threshold=0.2, nlu_threshold=0.7)
agent = Agent('domain.yml' , policies=[MemoizationPolicy(max_history=10), KerasPolicy(), fallback])
data = agent.load_data('stories.md')
agent.train(
				data,
				epochs = 500,
				batch_size = 50,
				validation_split = 0.2
)
agent.persist('models/dialogue')

ValueError: ignored